<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100"/>

# Cours TAL - Laboratoire 6
# Classification

**Objectif**

L’objectif de ce labo est de réaliser des expériences de classification de documents sous NLTK avec le
corpus de dépêches Reuters. Le labo est à effectuer en binôme. Le labo sera jugé sur la qualité des
expériences et sur la discussion des différentes options explorées. Vous devez remettre un notebook
Jupyter présentant vos choix, votre code, vos résultats et les discussions.


## 1. Données
les dépêches du corpus Reuters, tel qu’il est fourni par NLTK. Vous respecterez
notamment la division en données d’entraînement (train) et données de test.

In [19]:
# Imports et téléchargement du corpus
import nltk
from nltk.corpus import reuters
nltk.download('reuters')

!unzip /root/nltk_data/corpora/reuters.zip -d /root/nltk_data/corpora

nltk.download('stopwords')


[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


Archive:  /root/nltk_data/corpora/reuters.zip
replace /root/nltk_data/corpora/reuters/cats.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
# Exploration du corpus
print(reuters.categories())
print(len(reuters.fileids()))
print(reuters.words(categories=['grain']))

['acq', 'alum', 'barley', 'bop', 'carcass', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'cotton', 'cotton-oil', 'cpi', 'cpu', 'crude', 'dfl', 'dlr', 'dmk', 'earn', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'groundnut', 'groundnut-oil', 'heat', 'hog', 'housing', 'income', 'instal-debt', 'interest', 'ipi', 'iron-steel', 'jet', 'jobs', 'l-cattle', 'lead', 'lei', 'lin-oil', 'livestock', 'lumber', 'meal-feed', 'money-fx', 'money-supply', 'naphtha', 'nat-gas', 'nickel', 'nkr', 'nzdlr', 'oat', 'oilseed', 'orange', 'palladium', 'palm-oil', 'palmkernel', 'pet-chem', 'platinum', 'potato', 'propane', 'rand', 'rape-oil', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'rye', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'strategic-metal', 'sugar', 'sun-meal', 'sun-oil', 'sunseed', 'tea', 'tin', 'trade', 'veg-oil', 'wheat', 'wpi', 'yen', 'zinc']
10788
['CHINA', 'DAILY', 'SAYS', 'VERMIN', 'EAT', '7', '-', ...]


In [21]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
# Séparation des documents d'entrainement et de test
train_filenames= list(filter(lambda file: file.startswith("training"),reuters.fileids()))
test_filenames= list(filter(lambda file: file.startswith("test"),reuters.fileids()))

def prepareFile(filename):
  words = reuters.raw(filename)
  arrray_words = [word.lower() for word in word_tokenize(words) if word.isalpha()]
  return (arrray_words, reuters.categories(filename))

train_files = [prepareFile(train_filename) for train_filename in train_filenames]
test_files  = [prepareFile(test_filename) for test_filename in test_filenames]
# print(train_files[0])
# print(test_files[0])

## 2. Hyper-paramètres 
veuillez étudier au moins deux hyperparamètres. Pour chacun, veuillez
comparer au moins deux valeurs et indiquer laquelle fournit le meilleur score. Vous pourrez
choisir parmi les hyperparamètres suivants :
- options de prétraitement des textes : stopwords, lemmatisation, tout en minuscules.
- options de représentation : présence/absence de mots indicateurs, nombre de mots
indicateurs ; présence/absence/nombre de bigrammes, trigrammes ; autres traits :
longueur de la dépêche, rapport tokens/types.
- classifieurs et leurs paramètres : divers choix possibles (voir la documentation NLTK).

Nous avons choisis les hyperparamètres suivants : stopwords et les classifieurs NaiveBayesClassifier/DecisionTreeClassifier

In [23]:
# définir les fonctions pour les hyperparamètres
from nltk.corpus import stopwords
# stopwords
stops = set(stopwords.words('english'))

def remove_stop_words(words):
  return list([word for word in words if word not in stops])

stopwords_options = ["stopwords", "no_stopwords"]
classifier_options = ["NaiveBayes", "DecisionTree"]

fdist_stopwords = FreqDist([word for (words, cat) in train_files for word in words])
fdist_no_stopwords = FreqDist(remove_stop_words([word for (words, cat) in train_files for word in words]))
vocabulary = {
    "stopwords": [word for (word, count) in fdist_stopwords.most_common(2000)],
    "no_stopwords":[word for (word, count) in fdist_no_stopwords.most_common(2000)]
}
def create_embeddings(vocabulary, words):
  embedding = {}
  for word in vocabulary:
    embedding[word] = word in words
  return embedding

def embedded_files(vocabulary, files):
  return [(create_embeddings(vocabulary, file[0]), file[1]) for file in files]

train_set = {}
test_set = {}
for option in stopwords_options:
    train_set[option] = embedded_files(vocabulary[option], train_files)
    test_set[option] = embedded_files(vocabulary[option], test_files)


## 3. Entraînement et 4. Optimisation
Veuillez définir et entraîner trois classifieurs binaires : chacun prédit si une dépêche est
étiquetée ou non avec la catégorie respective. Le premier classifieur binaire sera pour
l’étiquette ‘money-fx’, le deuxième concernera ‘grain’, et le troisième sera pour ‘nat-gas’.
Pour chacun des classifieurs, optimisez les hyperparamètres sans toucher aux données de test
NLTK. Divisez les données d’entraînement NLTK en 80% train et 20% dev, et choisissez les
options qui donnent les meilleurs scores sur dev.

In [28]:
from nltk import NaiveBayesClassifier
import math
from random import shuffle

labels = ['money-fx', 'grain', 'nat-gas']

# Maybe separate respecting proportion :p
def separate_list(the_list, train_rate= 0.8):
  index = math.floor(len(the_list) * train_rate)
  shuffle(the_list)
  return (the_list[:index], the_list[index:])

def contains_label(data, label):
  return (data[0], label in data[1])

labelised_set = {
    "train": {},
    "dev": {},
    "test": {}
}

for label in labels:
  label_train_data = {}
  label_dev_data = {}
  label_test_data = {}
  for option in stopwords_options:
    labelised_train_files = [contains_label(train_file, label) for train_file in train_set[option]]
    (label_train_data[option], label_dev_data[option]) = separate_list(labelised_train_files, train_rate=0.8)
    label_test_data[option] = [contains_label(test_file, label) for test_file in test_set[option]]
  labelised_set["train"][label] = label_train_data
  labelised_set["dev"][label] = label_dev_data
  labelised_set["test"][label] = label_test_data


In [29]:
from nltk.metrics.scores import recall, precision, f_measure
def classifier_train(prepared_list, classifier_name, stopwords):
  train_list = prepared_list[stopwords]
  if classifier_name == "NaiveBayes":
    classifier = nltk.classify.NaiveBayesClassifier.train(train_list)
  elif classifier_name == "DecisionTree":
    classifier = nltk.classify.DecisionTreeClassifier.train(train_list)
  else:
    print("Unknown classifier")
    return
  return classifier

def classifier_score(labelised_list, classifier, stopwords):
  dev_list = labelised_list[stopwords]
  predicted = set()
  reference = set()
  for i, (data, label) in enumerate(dev_list):
    if label:
      reference.add(i)
    if classifier.classify(data):
      predicted.add(i)
  precision_score = precision(reference, predicted)
  recall_score = recall(reference, predicted)
  f1_score = f_measure(reference, predicted)
  return {"precision": precision_score, "recall": recall_score, "f1_score": f1_score}

In [30]:
scores = {}
for label in labels:
  label_scores = {}
  for classifier_type in classifier_options:
    classifier_type_score = {}
    for stopwords_option in stopwords_options:
      classifier = classifier_train(labelised_set["train"][label], classifier_type, stopwords_option)
      classifier_type_score[stopwords_option] = classifier_score(labelised_set["dev"][label], classifier, stopwords_option)
    label_scores[classifier_type] = classifier_type_score
  scores[label] = label_scores
print(scores)

{'money-fx': {'NaiveBayes': {'stopwords': {'precision': 0.4008438818565401, 'recall': 0.8482142857142857, 'f1_score': 0.5444126074498566}, 'no_stopwords': {'precision': 0.49444444444444446, 'recall': 0.7739130434782608, 'f1_score': 0.6033898305084746}}, 'DecisionTree': {'stopwords': {'precision': 0.821917808219178, 'recall': 0.5357142857142857, 'f1_score': 0.6486486486486486}, 'no_stopwords': {'precision': 0.8648648648648649, 'recall': 0.5565217391304348, 'f1_score': 0.6772486772486772}}}, 'grain': {'NaiveBayes': {'stopwords': {'precision': 0.32231404958677684, 'recall': 0.896551724137931, 'f1_score': 0.47416413373860183}, 'no_stopwords': {'precision': 0.3471502590673575, 'recall': 0.8170731707317073, 'f1_score': 0.48727272727272725}}, 'DecisionTree': {'stopwords': {'precision': 0.8765432098765432, 'recall': 0.8160919540229885, 'f1_score': 0.8452380952380952}, 'no_stopwords': {'precision': 0.9, 'recall': 0.7682926829268293, 'f1_score': 0.8289473684210525}}}, 'nat-gas': {'NaiveBayes': {

In [31]:
best_hyper_parameters = {}

for label in labels:
  best_classifier = ""
  best_stopwords_option = ""
  best_score = {"f1_score": -1}
  for classifier_option in classifier_options:
    for stopwords_option in stopwords_options:
      score = scores[label][classifier_option][stopwords_option]
      if score["f1_score"] > best_score["f1_score"]:
        best_score = score
        best_classifier = classifier_option
        best_stopwords_option = stopwords_option
  best_hyper_parameters[label] = {"stopwords": best_stopwords_option, "classifier": best_classifier}
  print("Les meilleurs hyperparamètres pour le label {} sont {} et {}".format(label, best_classifier, best_stopwords_option))
  print("precision: {}, rappel: {}, f_measure: {}".format(best_score["precision"], best_score["recall"], best_score["f1_score"]))

Les meilleurs hyperparamètres pour le label money-fx sont DecisionTree et no_stopwords
precision: 0.8648648648648649, rappel: 0.5565217391304348, f_measure: 0.6772486772486772
Les meilleurs hyperparamètres pour le label grain sont DecisionTree et stopwords
precision: 0.8765432098765432, rappel: 0.8160919540229885, f_measure: 0.8452380952380952
Les meilleurs hyperparamètres pour le label nat-gas sont DecisionTree et stopwords
precision: 0.8571428571428571, rappel: 0.5, f_measure: 0.631578947368421


## 5. Scores
Veuillez donner les scores de rappel, précision et f-mesure de chacun des trois classifieurs,
avec les meilleurs hyperparamètres, sur les données de test.

In [32]:
test_scores = {}
for label in labels:
  best_param = best_hyper_parameters[label]
  classifier = classifier_train(labelised_set["train"][label], best_param["classifier"], best_param["stopwords"])
  label_scores = classifier_score(labelised_set["test"][label], classifier, best_param["stopwords"])
  test_scores[label] = label_scores
print(test_scores)

{'money-fx': {'NaiveBayes': {'stopwords': {'precision': 0.4008438818565401, 'recall': 0.8482142857142857, 'f1_score': 0.5444126074498566}, 'no_stopwords': {'precision': 0.49444444444444446, 'recall': 0.7739130434782608, 'f1_score': 0.6033898305084746}}, 'DecisionTree': {'stopwords': {'precision': 0.821917808219178, 'recall': 0.5357142857142857, 'f1_score': 0.6486486486486486}, 'no_stopwords': {'precision': 0.8648648648648649, 'recall': 0.5565217391304348, 'f1_score': 0.6772486772486772}}}, 'grain': {'NaiveBayes': {'stopwords': {'precision': 0.32231404958677684, 'recall': 0.896551724137931, 'f1_score': 0.47416413373860183}, 'no_stopwords': {'precision': 0.3471502590673575, 'recall': 0.8170731707317073, 'f1_score': 0.48727272727272725}}, 'DecisionTree': {'stopwords': {'precision': 0.8765432098765432, 'recall': 0.8160919540229885, 'f1_score': 0.8452380952380952}, 'no_stopwords': {'precision': 0.9, 'recall': 0.7682926829268293, 'f1_score': 0.8289473684210525}}}, 'nat-gas': {'NaiveBayes': {

In [45]:
for label in labels:
  print("Score pour le label {} sur les données de test : ".format(label))
  print("precision: {}, rappel: {}, f_measure: {}".format(test_scores[label]["precision"], test_scores[label]["recall"], test_scores[label]["f1_score"]))
  print("Avec les hyperparamètres {} et {}".format(best_hyper_parameters[label]["classifier"], best_hyper_parameters[label]["stopwords"]))
  print("="* 20)

Score pour le label money-fx sur les données de test : 
precision: 0.7043478260869566, rappel: 0.45251396648044695, f_measure: 0.5510204081632653
Avec les hyperparamètres DecisionTree et no_stopwords
Score pour le label grain sur les données de test : 
precision: 0.9302325581395349, rappel: 0.8053691275167785, f_measure: 0.8633093525179857
Avec les hyperparamètres DecisionTree et stopwords
Score pour le label nat-gas sur les données de test : 
precision: 0.7857142857142857, rappel: 0.36666666666666664, f_measure: 0.5
Avec les hyperparamètres DecisionTree et stopwords


## 6. Classifieur multi-classe
Veuillez définir un quatrième classifieur multi-classe qui assigne une étiquette parmi quatre :
les trois choisies ci-dessus plus la catégorie ‘other’. Vous devrez nettoyer les données, car un
petit nombre de dépêches sont annotées avec plusieurs étiquettes : dans ce cas, gardez
seulement la première.

In [35]:
multi_labels = labels + ["other"]
monolabel_train_set = {}
monolabel_test_set = {}
for option in stopwords_options:
  monolabel_train_set[option] = [(words, [labels[0]]) for (words, labels) in train_set[option]]
  monolabel_test_set[option] = [(words, [labels[0]]) for (words, labels) in test_set[option]]


def labelised_or_other(data, labels, other_label):
  for label in labels:
    if label in data[1]:
      return (data[0], label)
  return (data[0], other_label)

multi_labelised_set = {
    "train":{},
    "dev": {},
    "test": {}
}
for option in stopwords_options:
    labelised_files = [labelised_or_other(train_file, labels, "other") for train_file in monolabel_train_set[option]]
    (multi_labelised_set["train"][option], multi_labelised_set["dev"][option]) = separate_list(labelised_files, train_rate=0.8)
    multi_labelised_set["test"][option] = [labelised_or_other(test_file, labels, "other") for test_file in monolabel_test_set[option]]


In [36]:
def multi_classifier_score(labelised_list, classifier, labels, stopwords):
  dev_list = labelised_list[stopwords]
  predicted = {}
  reference = {}
  labels_score = {}
  for label in labels:
    predicted[label] = set()
    reference[label] = set()
  for i, (data, label) in enumerate(dev_list):
    reference[label].add(i)
    predicted[classifier.classify(data)].add(i)
  for label in labels:
    precision_score = precision(reference[label], predicted[label])
    recall_score = recall(reference[label], predicted[label])
    f1_score = f_measure(reference[label], predicted[label])
    labels_score[label] = {"precision": precision_score, "recall": recall_score, "f1_score": f1_score}
  return labels_score

In [42]:
multi_scores = {}

for classifier_type in classifier_options:
    classifier_type_score = {}
    for stopwords_option in stopwords_options:
      classifier = classifier_train(multi_labelised_set["train"], classifier_type, stopwords_option)
      classifier_type_score[stopwords_option] = multi_classifier_score(multi_labelised_set["dev"], classifier, multi_labels, stopwords_option)
    multi_scores[classifier_type] = classifier_type_score
print(multi_scores)

{'NaiveBayes': {'stopwords': {'money-fx': {'precision': 0.17058823529411765, 'recall': 0.6304347826086957, 'f1_score': 0.26851851851851855}, 'grain': {'precision': 0.22413793103448276, 'recall': 0.7878787878787878, 'f1_score': 0.348993288590604}, 'nat-gas': {'precision': 0.0, 'recall': 0.0, 'f1_score': 0}, 'other': {'precision': 0.9779591836734693, 'recall': 0.8144119646498981, 'f1_score': 0.8887240356083087}}, 'no_stopwords': {'money-fx': {'precision': 0.21428571428571427, 'recall': 0.8048780487804879, 'f1_score': 0.3384615384615385}, 'grain': {'precision': 0.2962962962962963, 'recall': 0.7804878048780488, 'f1_score': 0.42953020134228187}, 'nat-gas': {'precision': 0.04878048780487805, 'recall': 0.25, 'f1_score': 0.08163265306122448}, 'other': {'precision': 0.9816147082334132, 'recall': 0.8387978142076503, 'f1_score': 0.9046040515653775}}}, 'DecisionTree': {'stopwords': {'money-fx': {'precision': 0.6666666666666666, 'recall': 0.043478260869565216, 'f1_score': 0.08163265306122448}, 'gra

In [53]:
best_multi_hyper_parameters = {}

def f1_measure_multi(multi_scores):
  total = 0
  for label in labels:
    score = multi_scores[label]["f1_score"] if multi_scores[label]["f1_score"] != None else 0
    total += score
  total /= len(labels)
  return total

best_classifier = ""
best_stopwords_option = ""
best_score = -1
best_label_scores = {}
for classifier_option in classifier_options:
  for stopwords_option in stopwords_options:
    current_multi_score = multi_scores[classifier_option][stopwords_option]
    if f1_measure_multi(current_multi_score) > best_score:
      best_label_scores = current_multi_score
      best_classifier = classifier_option
      best_stopwords_option = stopwords_option
best_multi_hyper_parameters = {"stopwords": best_stopwords_option, "classifier": best_classifier}
print("Les meilleurs hyperparamètres pour le multiclassifier sont {} et {}".format(best_classifier, best_stopwords_option))
for label in multi_labels:
  print("Scores pour le label {}".format(label))
  print("precision: {}, rappel: {}, f_measure: {}".format(best_label_scores[label]["precision"], best_label_scores[label]["recall"], best_label_scores[label]["f1_score"]))


Les meilleurs hyperparamètres pour le multiclassifier sont DecisionTree et no_stopwords
Scores pour le label money-fx
precision: 0.6666666666666666, rappel: 0.14634146341463414, f_measure: 0.24
Scores pour le label grain
precision: 0.9583333333333334, rappel: 0.5609756097560976, f_measure: 0.7076923076923076
Scores pour le label nat-gas
precision: None, rappel: 0.0, f_measure: None
Scores pour le label other
precision: 0.9598948060486522, rappel: 0.9972677595628415, f_measure: 0.9782244556113903


##7. Score
Veuillez donner les scores de rappel, précision et f-mesure de ce classifieur pour chacune des
trois étiquettes choisies. Comment les scores se comparent-ils à ceux des trois classifieurs
binaires ?

In [56]:
multi_classifier = classifier_train(multi_labelised_set["train"], best_multi_hyper_parameters["classifier"], best_multi_hyper_parameters["stopwords"])
multi_test_scores = multi_classifier_score(multi_labelised_set["test"], classifier, multi_labels, best_multi_hyper_parameters["stopwords"])

In [57]:
multi_test_scores

{'grain': {'f1_score': 0.7272727272727273,
  'precision': 0.7878787878787878,
  'recall': 0.6753246753246753},
 'money-fx': {'f1_score': 0.2644628099173554,
  'precision': 0.64,
  'recall': 0.16666666666666666},
 'nat-gas': {'f1_score': None, 'precision': None, 'recall': 0.0},
 'other': {'f1_score': 0.9755250824509634,
  'precision': 0.9596994535519126,
  'recall': 0.9918813978115072}}

In [58]:
print("Les scores de test du multiclassifier sont :")
for label in multi_labels:
  print("Scores pour le label {}".format(label))
  print("precision: {}, rappel: {}, f_measure: {}".format(multi_test_scores[label]["precision"], multi_test_scores[label]["recall"], multi_test_scores[label]["f1_score"]))
  print("="* 20)
print("Avec les hyperparamètres {} et {}".format(best_classifier, best_stopwords_option))


Les scores de test du multiclassifier sont :
Scores pour le label money-fx
precision: 0.64, rappel: 0.16666666666666666, f_measure: 0.2644628099173554
Scores pour le label grain
precision: 0.7878787878787878, rappel: 0.6753246753246753, f_measure: 0.7272727272727273
Scores pour le label nat-gas
precision: None, rappel: 0.0, f_measure: None
Scores pour le label other
precision: 0.9596994535519126, rappel: 0.9918813978115072, f_measure: 0.9755250824509634
Avec les hyperparamètres DecisionTree et no_stopwords


Pour tous les labels, les classifiers dédiés sont meilleurs que le multiclassifier, ce qui parait logique vu qu'il est moins spécialisé.

On notera que les hyperparmètres pour le multiclassifier ont été choisi en prenant le f1_score moyen "Macro". Ceci explique la non-performance pour le label "nat-gas"

##8. Conclusion

On remarque que le model "DecisionTree" est très souvent plus efficace, pour autant que les données soient présentes en quantité suffisante. Par contre le temps d'entrainement est lui beaucoup plus long.

On remarque peu de différence avec ou sans les stopwords, même si souvent les enlever est meilleur.

Un vocabulaire trop petit péjore également beaucoup les performances. Nous avions commencé par un vocabulaire de taille 500, mais le label "nat-gas" en patissait beaucoup.